# Real Estate Market Analysis (Tbilisi, Georgia)
Scrape real estate data and analyze using multiple libraries in Python for data visualization. 
Multiples steps, along with the codes used for each of them will be explained below.

## 00. Scraping real estate data

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

## add complete url to scrape data from 

url = ''

## create new dictionary

d = {'key':'value'}
print (d)

## update dictionary

d['new key'] = 'new value'
print (d)

## do the actual scraping using html tags from the webpage

npo_rentals = {}
rental_no = 0
while True:
    
    response = requests.get(url)
    data = response.text
    soup = BeautifulSoup(data, 'html.parser')
    rentals = soup.find_all('div',{'class':'wrapper'})
    
    for rental in rentals:
        title_tag = rental.find('h5',{'class':'card-title'})
        title = title_tag.text if title_tag else "N/A"
        location_tag = rental.find('div',{'class':'address'})
        location = location_tag.text if location_tag else "N/A"
        priceusd_tag = rental.find('b',{'class':'item-price-usd'})
        priceusd = priceusd_tag.text if priceusd_tag else "N/A"
        size_tag = rental.find('div',{'class':'item-size'})
        size = size_tag.text if size_tag else "N/A"
        date_tag = rental.find('div',{'class':'statement-date'})
        date = date_tag.text if date_tag else "N/A"
        rental_no+=1
        npo_rentals[rental_no] = [title, location, priceusd, size, date] ## sort the scraped data by different variables
        
        
        print('Rental:', title, '\nLocation', location, '\nPrice_USD:', priceusd, '\nSize:', size, '\nDate:', date, '\n---')
    url_tag = soup.find('li', {'class':'page-item number normal-item'})
    if url_tag.get('href'):
        url='' + url_tag.get('href') ## add the initial url of the webpage between brackets
        print(url)
    else:
        break 
print("Total Rentals:", rental_no)

## order scraped data by column to be recored into the .csv file

npo_rentals_df = pd.DataFrame.from_dict(npo_rentals, orient = 'index', columns = ['Rental', 'Location', 'Price_USD', 'Size', 'Date' ])

## save to .csv file

npo_rentals_df.to_csv('npo_rental.csv')

Note: this code scrapes data from only one page. There are much complex scraping solutions out there that will automatically scrape data through all the pages available. I used this code for every page separately, by putting them into the same file and only modifying url and output file names.

## 01. Combining files into one .csv file.

This code serves to combine all the exported data from multiple pages into one .csv file.

In [ ]:
import os
import glob
import pandas as pd
os.chdir("## add the location where you would like to save the data here")

extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])

#export to csv
combined_csv.to_csv( "combined_file.csv", index=False, encoding='utf-8-sig')

Now that we have scraped data combined into one .csv file, we can proceed to explore the data, analyze and plot.

## 1. Monthly Rental
Analysis of per month apartment rental market (prices in GEL)

### 1.1. Outliers in the dataset
Detecting outliers for more accurate analysis

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas import read_csv

In [ ]:
df=read_csv('tbilisi_rent_monthly_original.csv')

In [ ]:
df

In [ ]:
plt.hist(df.Price_GEL)
plt.show()

This histogram shows that some 50 000 + observations from the dataset are concentrated within one bar but the scale goes up until 5 000 000, meaning that there must be statistically insignificant number of observations outside the histogram bar that are not visibe on the plot. To find out the exact number of outliers and their values, lower and upper bound must be set as shown below.

In [ ]:
lower_bound = 0.05
upper_bound = 0.95
res = df.Price_GEL.quantile([lower_bound, upper_bound])
res

...this means that any value below lower_bound and above upper_bound GEL are outliers and will have excessive impact on the final results. For more accurate analysis, one can remove any values below or above them, or keep on or the other, or both. For a statically more accurate analysis, it is advised to remove them. You can keep the original dataset and create a new one, with outliers removed and do the JoyPlot using that new dataset.

### 1.2. BoxPlot
Creating boxplot to visualize outliers

In [ ]:
import pandas as pd
import seaborn as sns

In [ ]:
rent = pd.read_csv('tbilisi_rent_monthly_original.csv')
rent.head()

In [ ]:
ax = sns.boxplot(x='District', y= 'Price_GEL', data=rent)

### 1.3. JoyPlot
Plotting data from corrected dataset, without outliers
#### 1.3.1. Rent price
GEL

In [ ]:
import joypy
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import cm

In [ ]:
rent = pd.read_csv('tbilisi_rent_monthly_corrected.csv')

In [ ]:
rent = rent[['District','Price_GEL']]
rent.head()

In [ ]:
fig, ax = joypy.joyplot(data=rent, 
                        by = 'District',
                        column=['Price_GEL'],
                        labels = ['გლდანი','დიდუბე','ვაკე','ისანი','კრწანისი','მთაწმინდა','ნაძალადევი','საბურთალო','სამგორი','ჩუღურეთი'],
                        colormap = plt.cm.tab20, 
                        alpha = 0.5, 
                        legend = False)

plt.title('ქირავდება ბინა', y = 0.98, x = 0.39, size = 16, fontweight='bold')
fig.suptitle('თვიური', y = 0.95, size = 14)
plt.xlabel('ლარი', size = 8, fontweight='bold')

fig.set_size_inches(12, 8)

In [ ]:
fig.savefig('tbilisi_monthly_rent.pdf')

ESSENTIALLY, REPEAT ALL THE PREVIOUS STEPS WITH OTHER TYPES OF RENTAL DATA, SUCH AS SALES, LEASE, RENT PER DAY, ETC.

# 3. Aggregating data

As the scraped data contains more than 50k records, obviously, there is a need to aggregate that information to clearly identify major trends there.

Various tasks are needed to perform on this stage:
1. Aggregate rental data on district level. 
2. Then real estate chracteristics, such as size or price (for this an outlier analysis is needed) will be grouped. 
3. All the aggregated and grouped data will be moved to a new dataframe and exported as a .csv file.

## 3.1. Grouping aggregated data
by rental estate type (new or old apartment), district and price (USD/GEL). size outliers analysis will be performed in chapter 3.2 and aggregated in chapter 3.3.

In [ ]:
import pandas as pd
import numpy as np
from pandas import read_csv

In [ ]:
df=read_csv('tbilisi_rent_monthly_corrected.csv')

In [ ]:
## simple stats to explore dataset

count1 = df['ID'].count()
count2 = df['Rental'].count()

mean1 = df['Price_USD'].mean()
mean2 = df['Price_GEL'].mean()

max1 = df['Price_USD'].max()
max2 = df['Price_GEL'].max()
max3 = df['Size_sq_m'].max()

min1 = df['Price_USD'].min()
min2 = df['Price_GEL'].min()
min3 = df['Size_sq_m'].min()

## grouping

groupby_count1 = df.groupby(['District']).count() # total number of observations by district
groupby_count2 = df.groupby(['Rental','District']).count() # total number of observations by rental estate (new or old) type and district 
groupby_mean1 = df.groupby(['Rental','District'])['Price_USD','Price_GEL'].mean() # mean of rental price by rental estate (new or old) type and district 

print ('Number of observations: ' + str(count1)) # returns the total number of observations from the dataset
print ('Mean rent GEL: ' + str(mean2)) # returns mean price in GEL 
print ('count total \n' + str(groupby_count1)) # returns groupby_count1 values
print ('count total by district and rental type \n' + str(groupby_count2)) # returns groupby_count2 values
print ('mean price by rental estate type and district \n' + str(groupby_mean1)) # returns groupby_mean1 values

These values will be added to the new dataframe and create a .csv file with aggregated data (Chapter 6.4). But before that, outliers analysis is needed (size_sq_m or rent price variables). as an example, let's find the outliers in the size variable.

## 3.2. Outliers in size_sq_m variable

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas import read_csv

In [ ]:
df=read_csv('tbilisi_rent_monthly_size.csv')

In [ ]:
plt.hist(df.Size_sq_m)
plt.show()

In [ ]:
lower_bound = 0.05
upper_bound = 0.95
res = df.Size_sq_m.quantile([lower_bound, upper_bound])
res

## 3.3. Grouping data by real estate type, district and size_sq_m 

In [ ]:
import pandas as pd
import numpy as np
from pandas import read_csv

In [ ]:
df=read_csv('tbilisi_rent_monthly_size_corrected.csv')

In [ ]:
## grouping
## mean of rental price by rental estate (new or old) type, district and size_sq_m

groupby_mean_size = df.groupby(['Rental','District'])['Size_sq_m'].mean() 
print ('mean price by rental estate type and district \n' + str(groupby_mean_size)) returns groupby_mean_size values

Now, with the outliers analysis done, and new dataset loaded with corrected information on size_sq_m variable, a new dataframe will be created and exported to .cvs file.

In [ ]:
from pandas import DataFrame

rent = {'District': ['საბურთალო', 'ვაკე', 'დიდუბე', 'მთაწმინდა', 'კრწანისი', 'ნაძალადევი', 'ჩუღურეთი', 'გლდანი', 'ისანი', 'სამგორი'],
        'Total': [26108, 8551, 4770, 4284, 2001, 1803, 1619, 1396, 893, 62],
        'NUM_NEW': [21410, 7093, 3602, 2824, 1655, 1572, 817, 1247, 712, 56],
        'NUM_OLD':[4698, 1457, 1168, 1461, 346, 231, 802, 149, 181, 6],
        'MEAN_USD_NEW':[498, 684, 382, 704, 503, 324, 516, 306, 450, 303],
        'MEAN_USD_OLD':[330, 463, 305, 492, 405, 277, 401, 261, 324, 251],
        'MEAN_GEL_NEW':[1444, 1985, 1109, 2043, 1460, 942, 1498, 889, 1307, 880],
        'MEAN_GEL_OLD':[957, 1344, 885, 1427, 1174, 805, 1163, 758, 939, 729],        
        'MEAN_SIZE_NEW':[76, 88, 70, 89, 79, 64, 75, 62, 70, 61],
        'MEAN_SIZE_OLD':[66, 77, 70, 69, 69, 65, 66, 70, 61, 60]}

df = DataFrame(rent, columns=['District', 'Total', 'NUM_NEW', 'NUM_OLD', 'MEAN_USD_NEW', 'MEAN_USD_OLD', 'MEAN_GEL_NEW', 'MEAN_GEL_OLD', 'MEAN_SIZE_NEW', 'MEAN_SIZE_OLD'])

export_csv = df.to_csv (r'/Users/gkankia/Desktop/Tbilisi Real Estate Market/agg_rent_monthly.csv', index=True, header=True, encoding='utf-8-sig') #Don't forget to add '.csv' at the end of the path

print (df)

# 4. Visualization of aggregated data

After the aggregation, data will now be plotted in a visually understanding manner

## 4.1. Double Doughnut pie

This chart shows the number of rental properties by type and district

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
# Data to plot

mpl.rcParams['font.size'] = 12
labels_group = ['საბურთალო', 'ვაკე','დიდუბე','მთაწმინდა','კრწანისი','ნაძალადევი','ჩუღურეთი','გლდანი','ისანი','სამგორი']
sizes_group = [261,85,47,42,20,18,16,13,8,0.6]
labels_sub = ['1444','ახალი','ძველი','ახალი','ძველი','ახალი','ძველი','ახალი','ძველი','ახალი','ძველი','ახალი','ძველი','ახალი','ძველი','ახალი','ძველი','ახალი','ძველი','ახალი']
sizes_sub = [214,47,70,15,36,11,28,14,17,3,15,3,8,8,12,1,7,1,0.5,0.1]
colors_group = ['#081d58','#253494','#225ea8','#1d91c0','#4eb3d3','#41b6c4','#7fcdbb','#c7e9b4','#edf8b1','#ffffd9']
colors_sub = ['#92c5de','#d1e5f0']

# Plot

plt.pie(sizes_group, labels=labels_group, radius=1, colors=colors_group, labeldistance=1, wedgeprops=dict(width=1.5, edgecolor='white'), counterclock=False, startangle=90, frame=True, rotatelabels=True)
plt.pie(sizes_sub, colors=colors_sub, radius=0.8, counterclock=False, startangle=90, wedgeprops=dict(width=1, edgecolor='white'))
centre_circle = plt.Circle((0,0),0.6,color='#d6604d', fc='white',linewidth=5)
fig = plt.gcf()
fig.gca().add_artist(centre_circle)
plt.axis('equal')
plt.title('გასაქირავებელი უძრავი ქონების განაწილება', size = 16, y = 1, x = 0.5, fontweight='bold') 
plt.legend(loc='center left')
plt.tight_layout()
plt.show()
fig.set_size_inches(12, 8)

In [ ]:
fig.savefig('tbilisi_rent_bar_2.pdf') ## this function will save the output as pdf file.

## 4.2. Dumbbell Plot
### 7.2.1. Difference of rental price

This chart shows the difference in rental prices between old and newly-built apartments.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('agg_rent_monthly_1.csv')

In [ ]:
ordered_df = df.sort_values(by='DistrictID') #this column DistrictID was added to the file agg_rent_monthly.csv created in step 6.3 
my_range=range(1,len(df.index)+1)

In [ ]:
fig, ax = plt.subplots()
    
rect = ax
rect.set_facecolor('white') 

# setting plot background color. 
# Add the previous two lines of code if you want other colors as a background rather then white, which is default

plt.hlines(y=my_range, xmin=ordered_df['MEAN_GEL_OLD'], xmax=ordered_df['MEAN_GEL_NEW'], color='#7bccc4', alpha=1, linewidth=2, zorder=2, facecolor='#d5de9c')
plt.scatter(ordered_df['MEAN_GEL_OLD'], my_range, color='#a8ddb5', alpha=1, label='ძველი აშენებული', s=120, zorder=3)
plt.scatter(ordered_df['MEAN_GEL_NEW'], my_range, color='#1d91c0', alpha=1, label='ახალი აშენებული', s=120, zorder=3)
plt.legend(fontsize=12)

# Add title, axis, names and customize further

plt.yticks(my_range, ordered_df['District'])
plt.title('ძველი და ახალი აშენებული ბინის \n საშუალო ქირა (თვეში)', size = 16, fontweight='bold', loc='center')
plt.xlabel('ლარი', size = 8, fontweight='bold')
plt.gca().invert_yaxis() # by default order was be from bottom to top
plt.grid(axis='y', color='#e0f3db', linestyle='-', linewidth=1, zorder=1)
fig.set_size_inches(12, 8)

In [ ]:
fig.savefig('tbilisi_rent_dumbbell_plot.pdf')

### 7.2.2. Difference of rental apartment size

This chart shows the difference in living space sizes (sq.m.) between old and newly-built apartments.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('agg_rent_monthly.csv')

In [ ]:
ordered_df = df.sort_values(by='DistrictID') #this column DistrictID was added to the file agg_rent_monthly.csv created in step 6.3 
my_range=range(1,len(df.index)+1)

In [ ]:
fig, ax = plt.subplots()
    
rect = ax
rect.set_facecolor('white') # setting plot background color
plt.hlines(y=my_range, xmin=ordered_df['MEAN_SIZE_OLD'], xmax=ordered_df['MEAN_SIZE_NEW'], color='#f768a1', alpha=1, linewidth=2, zorder=2, facecolor='#d5de9c')
plt.scatter(ordered_df['MEAN_SIZE_OLD'], my_range, color='#fcc5c0', alpha=1, label='ძველი აშენებული', s=120, zorder=3)
plt.scatter(ordered_df['MEAN_SIZE_NEW'], my_range, color='#dd3497', alpha=1, label='ახალი აშენებული', s=120, zorder=3)
plt.legend(fontsize=12)

# Add title, axis, names and customize further
plt.yticks(my_range, ordered_df['District'])
plt.title('ძველი და ახალი აშენებული ბინის \n საერთო ფართი (კვ.მ.)', size = 16, fontweight='bold', loc='center')
plt.xlabel('კვ.მ.', size = 8, fontweight='bold')
plt.gca().invert_yaxis() # by default order was be from bottom to top
plt.grid(axis='y', color='#fde0dd', linestyle='-', linewidth=1, zorder=1)
fig.set_size_inches(12, 8)

In [ ]:
fig.savefig('tbilisi_rent_size_dumbbell_plot.pdf')